In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

# Mute warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_feather('./dataset/cleaned_train.feather')
train.head()

In [ ]:
print(np.exp(train.SalePrice.median()))
print(np.exp(train.SalePrice.max()))

In [ ]:
test = pd.read_feather('./dataset/cleaned_test.feather')
test.head()

In [ ]:
train[train.isin([np.nan, -np.nan, np.inf, -np.inf]).any(1)]

In [ ]:
train_target = train.pop("SalePrice")

In [ ]:
print(np.exp(train_target.median()))
print(np.exp(train_target.max()))

## Feature selection

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.feature_selection import VarianceThreshold, mutual_info_regression, SelectKBest, f_regression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

### Constant features

In [ ]:
sel = VarianceThreshold(threshold=0.01)
sel.fit(train)

In [ ]:
print(
    len([
        x for x in train.columns
        if x not in train.columns[sel.get_support()]
    ]))

train_sel = [x for x in train.columns if x not in train.columns[sel.get_support()]]

train = train.loc[:, ~train.columns.isin(train_sel)]
test =  test.loc[:, ~test.columns.isin(train_sel)]

### Univariate features 

In [ ]:
# fs = SelectKBest(score_func=f_regression, k=10)
# df_fs = fs.fit_transform(df, df_target)
# df_fs.shape

In [ ]:
# fs.get_feature_names_out()

### Redundant features

In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:  # we are interested in absolute coefficient value
                col_name = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(col_name)
    return col_corr

corr_features = correlation(train, 0.75)  # 0.5

train = train.loc[:, ~train.columns.isin(corr_features)]
test =  test.loc[:, ~test.columns.isin(corr_features)]

In [ ]:
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# sfs1 = SFS(RandomForestRegressor(), 
#            k_features=10, 
#            forward=True, 
#            floating=False, 
#            verbose=2,
#            scoring='r2',
#            cv=3)

# sfs1 = sfs1.fit(np.array(df_corr), df_target)

In [ ]:
# sfs1.k_feature_idx_

In [ ]:
# df.columns[list(sfs1.k_feature_idx_)]

### Feature Utility Scores

In [ ]:
def make_mi_scores(X, y):
    mi_scores = mutual_info_regression(X, y, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")
    
    
def drop_uninformative(df, mi_scores):
    return df.loc[:, df.columns.isin(mi_scores[mi_scores > 0.0].index)]

In [ ]:
mi_scores = make_mi_scores(train, train_target)
mi_scores

In [ ]:
train = drop_uninformative(train, mi_scores)
test  = drop_uninformative(test, mi_scores)

## Model

In [ ]:
import optuna
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import RobustScaler
from xgboost import XGBRegressor

In [ ]:
transformer = RobustScaler().fit(train)

train_scaled = transformer.transform(train)
test_scaled = transformer.transform(test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    train_scaled,
    train_target,
    test_size=0.3,
    random_state=0
)

### Cross-Validation

In [ ]:
n_folds = 5

scorer = make_scorer(mean_squared_error, greater_is_better=False)

def rmse_CV(model, X, y):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring ="neg_mean_squared_error", cv=kf))
    return (rmse)

We are checking how model is performing on our own data

What is the difference between test data and validation data

In [ ]:
def prediction(model, X, y, test_X, test_y):
    m = model.fit(X, y)
    pred_train = m.predict(X)
    pred_test  = m.predict(test_X)
    print('rmse on train set', rmse_CV(m, X, y).mean())
    print('rmse on test set', rmse_CV(m, test_X, test_y).mean())
    
    return m, pred_train, pred_test

def residuals_plot(pred_train, train_y, pred_test, test_y, title):
    plt.scatter(pred_train, pred_train - train_y, c="blue",  label="Training data")
    plt.scatter(pred_test, pred_test - test_y, c="green",  label="Validation data")

    plt.title(title)
    plt.xlabel("Predicted values")
    plt.ylabel("Residuals")
    plt.legend(loc="upper left")
    plt.hlines(y=0, xmin=pred_train.min(), xmax=pred_train.max(), color="red")
    plt.show()
    
def linear_plot(pred_train, train_y, pred_test, test_y, title):
    plt.scatter(pred_train, train_y, c="blue",  label="Training data")
    plt.scatter(pred_test, test_y, c="green",  label="Validation data")

    plt.title(title)
    plt.xlabel("Predicted values")
    plt.ylabel("Real values")
    plt.legend(loc="upper left")
    plt.plot([pred_train.min(), pred_train.max()], [pred_train.min(), pred_train.max()], c="red")
    plt.show()   

### Linear Regression

In [ ]:
# lr, train_lr, test_lr = prediction(LinearRegression(), X_train, y_train, X_test, y_test)
# residuals_plot(train_lr, y_train, test_lr, y_test, "Linear regression residuals")
# linear_plot(train_lr, y_train, test_lr, y_test, "Linear regression real values")

Les points residuelles sont dispersés de manière **random** autour de l'axe horizontal, on peut utiliser un modèle linéaire.

Regularization is a very useful method to handle collinearity, filter out noise from data, and eventually prevent overfitting.

The concept behind regularization is to introduce additional information (bias) to penalize extreme parameter weights.

Ridge and Lasso Regression are types of Regularization techniques

### Ridge Regression

In [ ]:
# ridge, train_ridge, test_ridge = prediction(
#     RidgeCV(alphas=[0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 3, 6, 10, 30, 60]), X_train, y_train, X_test, y_test
# )

# alpha = ridge.alpha_
# print('Best alpha', alpha)

# ridge, train_ridge, test_ridge = prediction(
#     RidgeCV(alphas=[alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
#                           alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
#                           alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4], cv=5), 
#     X_train, y_train, X_test, y_test
# )

# alpha = ridge.alpha_
# print("Best alpha :", alpha)

# residuals_plot(train_ridge, y_train, test_ridge, y_test, "Ridge regression residuals")
# linear_plot(train_ridge, y_train, test_ridge, y_test, "Ridge regression real values")

### LASSO Regression

In [ ]:
# lasso, train_lasso, test_lasso = prediction(
#     LassoCV(alphas = [1, 0.1, 0.001, 0.0005]), X_train, y_train, X_test, y_test
# )

# residuals_plot(train_lasso, y_train, test_lasso, y_test, "Lasso regression residuals")
# linear_plot(train_lasso, y_train, test_lasso, y_test, "Lasso regression real values")

In [ ]:
# coef_lasso = pd.Series(lasso.coef_, index=train.columns)

# print("Lasso picked " + str(sum(coef_lasso != 0)) + " variables and eliminated the other " +  str(sum(coef_lasso == 0)) + " variables")

In [ ]:
# imp_coef = pd.concat([coef_lasso.sort_values().head(10),
#                      coef_lasso.sort_values().tail(10)])

In [ ]:
# plt.figure(figsize=(10,10))
# imp_coef.plot(kind = "barh")
# plt.title("Coefficients in the Lasso Model");

Pour les coeffs négatifs, voir unbalanced categorical variables

### ElasticNet

In [ ]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009]
l1ratio = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [ ]:
elasticnet, train_elasticnet, test_elasticnet = prediction(
    ElasticNetCV(max_iter=1e7, alphas=alphas, l1_ratio=l1ratio), X_train, y_train, X_test, y_test
)

residuals_plot(train_elasticnet, y_train, test_elasticnet, y_test, "ElasticNet regression residuals")
linear_plot(train_elasticnet, y_train, test_elasticnet, y_test, "ElasticNet regression real values")

### XGBoost

In [ ]:
def objective(trial):
    xgb_params = dict(
        max_depth=trial.suggest_int("max_depth", 2, 10),
        learning_rate=trial.suggest_float("learning_rate", 1e-4, 1e-0, log=True),
        n_estimators=trial.suggest_int("n_estimators", 100, 6000),
        min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
        subsample=trial.suggest_float("subsample", 0.2, 1.0),
        reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e2, log=True),
        reg_lambda=trial.suggest_float("reg_lambda", 1e-4, 1e2, log=True),
    )
    xgb = XGBRegressor(**xgb_params)
    return rmse_CV(xgb, X_train, y_train).mean()

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)
xgb_params = study.best_params

In [ ]:
xgb = XGBRegressor(**xgb_params)

xgb.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)], verbose=False)

pred_test_xgb = np.exp(xgb.predict(X_test))
pred_xgb = np.exp(xgb.predict(test_scaled))

In [ ]:
# Plot residuals
plt.scatter(pred_test_xgb, pred_test_xgb - np.exp(y_test), c="blue",  label="Validation data")

plt.title("XGBoost regression")
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.legend(loc="upper left")
plt.hlines(y=0, xmin=pred_test_xgb.min(), xmax=pred_test_xgb.max(), color="red")
plt.show()

In [ ]:
results = pd.concat([test, pd.DataFrame({"SalePrice": pred_xgb})], axis=1)
results.head()

In [ ]:
q1 = results['SalePrice'].quantile(0.005)
q2 = results['SalePrice'].quantile(0.995)

results['SalePrice'] = results['SalePrice'].apply(lambda x: x if x > q1 else x*0.77)
results['SalePrice'] = results['SalePrice'].apply(lambda x: x if x < q2 else x*1.1)

In [ ]:
print(results['SalePrice'].median())
print(results['SalePrice'].max())

In [ ]:
q1 = results['SalePrice'].quantile(0.005)
q2 = results['SalePrice'].quantile(0.995)

results['SalePrice'] = results['SalePrice'].apply(lambda x: x if x > q1 else x*0.77)
results['SalePrice'] = results['SalePrice'].apply(lambda x: x if x < q2 else x*1.1)

In [ ]:
print(results['SalePrice'].median())
print(results['SalePrice'].max())

In [ ]:
%reset -f